## Prospect Theory and Stock Returns : An Empirical Test 

### 1. us ticker 수집 및 ticker 과거 주가 

In [1]:
# !pip install yfinance

  Running setup.py install for multitasking ... done
  Running setup.py install for yfinance ... done
You are using pip version 18.1, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import yfinance as yf
import pandas as pd
import os
import pickle

#### ticker 출처 : "https://stockanalysis.com/stocks/"

In [76]:
#### ticker를 뽑아줌`

import requests

get_ticker_url = "https://stockanalysis.com/stocks/"

get_ticker = requests.get(get_ticker_url)

from lxml import html
get_ticker = html.fromstring(get_ticker.content)
ticker_list = get_ticker.xpath('/html/body/div[3]/div/div[2]/main/article/div/div/ul/li')

ticker = []
for t in ticker_list:
    raw = t.text_content()
    raw_list = raw.split('-')
    key = raw_list[0].strip()
    ticker.append(key)

In [ ]:
#### ticker별 과거 주가 저장

if 'us_ticker' in os.listdir(os.getcwd()):
    pass
else:
    os.mkdir('us_ticker')

for t in ticker:
    temp = yf.Ticker(t)
    temp_df = temp.history(period='max')
    
    with open("us_ticker/{}.pickle".format(t), 'wb') as f:
        pickle.dump(temp_df, f)


--------------
-------------

### 2. 월간 수익률 구하기

In [1]:
import os
import pandas as pd
import numpy as np
import pickle

In [2]:
# resample('M') - 같은 달끼리 묶어줌(groupby)
# resample('M').first() / last() - 같은 달 내의 첫쨰 값 / 마지막 값

# 월간 수익률(해당 년 월)
def mon_return(s):
    first = s.resample('M').first()
    last = s.resample('M').last()
    return (last - first) / first

In [ ]:
# 월간 수익률 저장

folder_path = os.path.join(os.getcwd(), 'us_ticker')
mon_return_dict = {}
for t in sorted(os.listdir(folder_path)):
    data = pd.read_pickle(os.path.join(folder_path, t))
    if len(data) == 0:     ### 데이터가 없는 건 제외
        pass
    else:
        name = t.split(".")[0]
        mon_return_dict[name] = mon_return(data['Close'])

with open('mon_return.pickle', 'wb') as f:
    pickle.dump(mon_return_dict, f)

In [3]:
mon = pd.read_pickle('mon_return.pickle')
mon_df = pd.DataFrame(mon)

In [5]:
mon_df

,A,AA,AAC,AACG,AACQ,AAIC,AAL,AAMC,AAME,AAN,...,ZTO,ZTS,ZUMZ,ZUO,ZVO,ZWRK,ZXAIY,ZYME,ZYNE,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
1962-01-31,NaN,-0.088290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-02-28,NaN,0.023405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-03-31,NaN,0.017286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-04-30,NaN,-0.034069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-05-31,NaN,-0.104308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31,0.027132,0.055886,NaN,0.034783,0.068342,0.138554,0.105116,0.025217,-0.046296,0.029875,...,0.014261,0.025657,-0.014205,0.203976,0.206107,NaN,0.181818,-0.135382,-0.261745,0.001488
2021-01-31,0.012896,-0.225806,NaN,0.110169,-0.003752,-0.029491,0.134831,-0.106695,0.112149,-0.099415,...,0.179323,-0.055633,0.206723,0.088561,0.110629,NaN,0.730769,-0.309373,0.049624,0.301873
2021-02-28,0.010262,0.304463,NaN,2.768657,0.012093,0.095628,0.243468,0.019222,0.926724,0.222717,...,-0.015178,-0.002185,0.063709,0.048455,-0.009652,NaN,-0.069767,0.018799,0.291667,-0.200881


### 3. 논문의 식 정의

In [6]:
# function 5 in page 8

def fun_5(x, alpha, lamda):
    if x >=0 :
        return x ** alpha
    else:
        return -lamda * ((-x) ** alpha)

In [7]:
# function 6 in page 8

def fun_6_plus(p, gamma):
    numerator = p ** gamma
    denominator = ((p ** gamma) + ((1-p) ** gamma)) ** (1/gamma)
    return numerator / denominator

def fun_6_minus(p, delta):
    numerator = p ** delta
    denominator = ((p ** delta) + ((1-p) ** delta)) ** (1/delta)
    return numerator / denominator

In [167]:
# function 8 in page 12

def fun_8(r):
    
    alpha = 0.88
    lamda = 2.25
    gamma = 0.61
    delta = 0.69
    
    r_drop = r.dropna()
    num_month = len(r_drop)
    sorted_r = np.sort(r_drop)
    m = np.argwhere(sorted_r > 0).min()  #### 정렬한 후 양수가 되는 시점이 m 값이 됨
    
    
    v_fun_5 = np.vectorize(fun_5)
    v_fun_6_plus = np.vectorize(fun_6_plus)
    v_fun_6_minus = np.vectorize(fun_6_minus)
    
    minus_index = np.arange(m)
    plus_index = np.sort(np.arange(num_month-m))[::-1] #### 내림차순 배열
    minus_r = np.array(sorted_r[:m])
    plus_r = np.array(sorted_r[m:])
    
    minus_v_r = v_fun_5(minus_r, alpha, lamda)
    plus_v_r = v_fun_5(plus_r, alpha, lamda)
    
    minus_w_r1 = v_fun_6_minus((minus_index+1)/num_month, delta)
    minus_w_r2 = v_fun_6_minus((minus_index)/num_month, delta)
    minus_w = minus_w_r1 + minus_w_r2
    
    plus_w_r1 = v_fun_6_plus((plus_index+1)/num_month, gamma)
    plus_w_r2 = v_fun_6_plus((plus_index)/num_month, gamma)
    plus_w = plus_w_r1 + plus_w_r2
    
    TK = np.dot(minus_v_r, minus_w) + np.dot(plus_v_r, plus_w)  #### 시그마를 내적으로 생각
    
    return TK

### 4. 논문 내용 확인
#### 4-1 2016년 1월 ~ 2020년 12월까지 월간 수익률(60개)를 바탕으로 TK 값 도출
#### 4-2 TK 값을 10분위 구간으로 나눠 2021년 1월의 실제 수익률과 TK 값과 비교

In [168]:
test_df = mon_df.loc['2016-01-31':'2020-12-31',:]
test_df = test_df.astype('float').dropna(axis = 1, how = 'any')
test_tk = test_df.apply(fun_8, axis = 0)

In [171]:
test_tk

A        0.264081
AA      -1.593292
AACG    -3.178079
AAIC    -0.916141
AAL     -1.768054
           ...   
ZTS      0.599934
ZUMZ    -0.934982
ZXAIY   -1.394822
ZYNE    -4.557949
ZYXI     1.162041
Length: 4108, dtype: float64

In [172]:
test_cut = pd.qcut(test_tk,10, range(10))

In [173]:
next_return_df = mon_df.loc['2021-01-31',:]

In [174]:
test_result = pd.concat((test_tk, test_cut, next_return_df), axis = 1, join = 'inner')
test_result.columns = ['tk_value', 'tk_tile', 'recent_return']

In [178]:
test_result

,tk_value,tk_tile,recent_return
A,0.264081,8,0.012896
AA,-1.593292,3,-0.225806
AACG,-3.178079,1,0.110169
AAIC,-0.916141,4,-0.029491
AAL,-1.768054,3,0.134831
...,...,...,...
ZTS,0.599934,9,-0.055633
ZUMZ,-0.934982,4,0.206723
ZXAIY,-1.394822,3,0.730769
ZYNE,-4.557949,0,0.049624


In [179]:
test_result_groupby = test_result.groupby(by = 'tk_tile')
test_result_groupby.mean()

,tk_value,recent_return
tk_tile,,
0,-6.185499,0.453674
1,-3.214088,0.257324
2,-2.147633,0.198345
3,-1.485628,0.103027
4,-1.038479,0.065857
5,-0.696394,0.038342
6,-0.421084,0.033731
7,-0.156593,0.018527
8,0.131593,0.015561


#### TK값이 커질수록 다음 달의 수익률이 작아지는 경향을 확인